In [1]:
import os
import cv2
from skimage import io
from tqdm import tqdm

In [3]:
import argparse
import os
import sys
import numpy as np
import csv
import cv2
import argparse 
import imageio
from skimage.filters import threshold_otsu, threshold_mean, threshold_local, threshold_isodata
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects
from scipy.ndimage.morphology import binary_fill_holes
from skimage.morphology import square
from skimage import morphology
from skimage.segmentation import clear_border
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import time

/var/folders/sd/z_kvtl2s6v93h4_t4j4zmpjm0000gn/T/ipykernel_55305/788962213.py:12: DeprecationWarning: Please use `binary_fill_holes` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import binary_fill_holes


In [4]:
def crop_and_save3(complete_image, centroid, shapeid, output_dir, size=160):
    """ Crops and saves a region from an image """
    
    nx_0 = max(int(centroid[0] - size/2),0)
    ny_0 = max(int(centroid[1] - size/2),0)
    nx_1 = min(nx_0 + size, complete_image.shape[1])
    ny_1 = min(ny_0 + size, complete_image.shape[0])
    #roi_type = shape['type'].split(':')[-1]
    roi_file = os.path.join(output_dir, str(shapeid)+'.png')
    cropped_image = complete_image[ny_0:ny_1, nx_0: nx_1,:]
    imageio.imwrite(roi_file, cropped_image)    
    



    
    
def rbc_segmentation(img, outputdir=None):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(5,5),0)
    ret, thresh = cv2.threshold(gray, 127, 255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
    sure_bg = cv2.dilate(opening,kernel,iterations=2)
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    ret, sure_fg = cv2.threshold(dist_transform,0.3*dist_transform.max(),255,0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers+1
    markers[unknown==255] = 0

    markers = cv2.watershed(img,markers)
    bw=(markers>1).astype(int)
    bw=binary_fill_holes(bw).astype(np.uint8)
    #img[markers == -1] = [255,0,0]
    bw_clean=morphology.remove_small_objects(bw.astype(bool), min_size=5000, connectivity=4).astype(np.uint8)
    rbc_gone = 255*remove_small_objects(bw_clean.astype(bool), min_size=17000, connectivity=4).astype(np.uint8)
    rbc_only = cv2.subtract(255*bw_clean, rbc_gone)
    #cv2.imwrite(outputdir +'.jpg', img)
    return rbc_only


def chop_thumbnails(image, output_dir, current_shapeid=0):
    shapeid = current_shapeid


    mp_masks=rbc_segmentation(image)
    
    output  = cv2.connectedComponentsWithStats(mp_masks, connectivity=8)
#    num_labels = output[0]
#    labels = output[1]
#    stats = output[2]
    centroids = output[3]    
#                
    for c in centroids:
        crop_and_save3(image, c, shapeid, output_dir)
        shapeid=shapeid+1
                
               
    return shapeid

Segment and isolate non sma RBCs

In [6]:
# Function for RBC segmentation and thumbnail extraction
def process_image(image, output_dir):
    # Perform RBC segmentation
    rbc_only = rbc_segmentation(image)
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Perform thumbnail extraction
    shapeid = chop_thumbnails(image, output_dir)
    
    return shapeid

# Define input and output directories
input_dir = '/Users/ezermoysis/Documents/UCL/Year end project/Malaria/FYP/Yu-SMA/non-sma-images'
output_dir = '/Users/ezermoysis/Documents/UCL/Year end project/Malaria/FYP/out_rbc'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Get a list of image files in the input directory
image_files = [file for file in os.listdir(input_dir) if file.endswith('.tiff') or file.endswith('.png')]

# Process each image
for image_file in tqdm(image_files, desc='Processing images'):
    # Read the image
    image_path = os.path.join(input_dir, image_file)
    image = cv2.imread(image_path)
    
    if image is not None:
        # Process the image
        shapeid = process_image(image, output_dir)
        
        print(f"Image {image_file} processed. Total shapes extracted: {shapeid}")

print("All images processed successfully.")


Processing images:   6%|▋         | 1/16 [00:01<00:29,  2.00s/it]

Image non-sma_6_EDOF_RGB.tiff processed. Total shapes extracted: 80


Processing images:  12%|█▎        | 2/16 [00:03<00:22,  1.62s/it]

Image non-sma_2_EDOF_RGB.tiff processed. Total shapes extracted: 135


Processing images:  19%|█▉        | 3/16 [00:04<00:19,  1.48s/it]

...

Image non-sma_4_EDOF_RGB.tiff processed. Total shapes extracted: 125


Processing images:  94%|█████████▍| 15/16 [00:21<00:01,  1.34s/it]

Image non-sma_3_EDOF_RGB.tiff processed. Total shapes extracted: 150


Processing images: 100%|██████████| 16/16 [00:22<00:00,  1.43s/it]

Image non-sma_7_EDOF_RGB.tiff processed. Total shapes extracted: 203
All images processed successfully.


Segment and isolate sma RBCs

In [7]:
# Function for RBC segmentation and thumbnail extraction
def process_image(image, output_dir):
    # Perform RBC segmentation
    rbc_only = rbc_segmentation(image)
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Perform thumbnail extraction
    shapeid = chop_thumbnails(image, output_dir)
    
    return shapeid

# Define input and output directories
input_dir = '/Users/ezermoysis/Documents/UCL/Year end project/Malaria/FYP/Yu-SMA/sma-images'
output_dir = '/Users/ezermoysis/Documents/UCL/Year end project/Malaria/FYP/rbc_sma'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Get a list of image files in the input directory
image_files = [file for file in os.listdir(input_dir) if file.endswith('.tiff') or file.endswith('.png')]

# Process each image
for image_file in tqdm(image_files, desc='Processing images'):
    # Read the image
    image_path = os.path.join(input_dir, image_file)
    image = cv2.imread(image_path)
    
    if image is not None:
        # Process the image
        shapeid = process_image(image, output_dir)
        
        print(f"Image {image_file} processed. Total shapes extracted: {shapeid}")

print("All images processed successfully.")


Processing images:   6%|▌         | 1/17 [00:01<00:23,  1.49s/it]

Image sma_11_EDOF_RGB.tiff processed. Total shapes extracted: 134


Processing images:  12%|█▏        | 2/17 [00:02<00:20,  1.39s/it]

Image sma_15_EDOF_RGB.tiff processed. Total shapes extracted: 116


Processing images:  18%|█▊        | 3/17 [00:03<00:16,  1.19s/it]

...

Image sma_13_EDOF_RGB.tiff processed. Total shapes extracted: 141


Processing images:  94%|█████████▍| 16/17 [00:17<00:01,  1.22s/it]

Image sma_14_EDOF_RGB.tiff processed. Total shapes extracted: 150


Processing images: 100%|██████████| 17/17 [00:18<00:00,  1.08s/it]

Image sma_10_EDOF_RGB.tiff processed. Total shapes extracted: 48
All images processed successfully.
